<a href="https://colab.research.google.com/github/adidror005/HomeAssignment/blob/main/EMA_ML_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas-ta
!pip install numpy==1.23.5
!pip install catboost


  Using cached catboost-1.2.8-cp311-cp311-manylinux2014_x86_64.whl.metadata (1.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 6.1 MB/s eta 0:00:00


In [ ]:
3

3

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_parquet('/content/drive/MyDrive/dollar_bars_AMD.parquet')

In [ ]:
import numpy as np
df['label_real']=np.log(df['close']/df['open']).shift(-1)
df['label']=np.sign(df['label_real'])


In [ ]:
df=df.drop_duplicates(subset=['ts_end']).drop('ts_start',axis=1)
df.set_index('ts_end',inplace=True)
df.sort_index()
df=df.astype(np.float64)

In [ ]:
df.open

,open
ts_end,
2017-01-03 14:31:12.032000,0.0
2017-01-03 14:31:52.190000,0.0
2017-01-03 14:33:45.154000,0.0
2017-01-03 14:35:03.433000,0.0
2017-01-03 14:36:11.302000,0.0
...,...
2025-05-08 19:54:40.227456,101.9
2025-05-08 19:56:52.507584,101.9
2025-05-08 19:58:34.941437,101.9


### Extract Features

In [ ]:
import pandas_ta as ta
import warnings

def apply_indicators(df):
    """Apply technical indicators cleanly to the given DataFrame."""
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=FutureWarning)

        # RSI
        df.ta.rsi(length=14, append=True,mamode="ewm",adjust=False)
        df.ta.rsi(length=840, append=True,mamode="ewm",adjust=False)
        df.ta.rsi(length=70, append=True,mamode="ewm",adjust=False)

        # PPO
        df.ta.ppo(fast=12, slow=26, signal=9, append=True,mamode="ewm",adjust=False)
        df.ta.ppo(fast=40, slow=200, signal=12, append=True,mamode="ewm",adjust=False)

        # Other Indicators
        df.ta.aroon(append=True,mamode="ewm",adjust=False)
        df.ta.bop(append=True,mamode="ewm",adjust=False)
        df.ta.stochrsi(append=True,mamode="ewm",adjust=False)

        df.ta.mfi(length=14, append=True,mamode="ewm",adjust=False)
        df.ta.mfi(length=70, append=True,mamode="ewm",adjust=False)

        df.ta.vwap(append=True,mamode="ewm",adjust=False)

        df.ta.kvo(append=True,mamode="ewm",adjust=False)


        df.ta.adx(length=14, append=True,mamode="ewm",adjust=False)
        df.ta.adx(length=70, append=True,mamode="ewm",adjust=False)
        df.ta.adx(length=840, append=True,mamode="ewm",adjust=False)

        df.ta.natr(length=14, append=True,mamode="ewm",adjust=False)
        df.ta.natr(length=70, append=True,mamode="ewm",adjust=False)
        df.ta.natr(length=840, append=True,mamode="ewm",adjust=False)

        #df.ta.sqz_indicator(append=True)

        df.ta.adosc(append=True,mamode="ewm",adjust=False)
        df.ta.pvr(append=True,mamode="ewm",adjust=False)
        df.ta.pvt(append=True,mamode="ewm",adjust=False)
        df.ta.chop(append=True,mamode="ewm",adjust=False)
        df.ta.rsx(append=True,mamode="ewm",adjust=False)
        df.ta.rvgi(append=True,mamode="ewm",adjust=False)
        df.ta.uo(append=True,mamode="ewm",adjust=False)
        df.ta.tsi(append=True,mamode="ewm",adjust=False)

    # Optional: enforce float64 everywhere
    return df
apply_indicators(df)

,open,high,low,close,volume,dollar_amount,label_real,label,RSI_14,RSI_840,...,ADOSC_3_10,PVR,PVT,CHOP_14_1_100,RSX_14,RVGI_14_4,RVGIs_14_4,UO_7_14_28,TSI_13_25_13,TSIs_13_25_13
ts_end,,,,,,,,,,,,,,,,,,,,,
2017-01-03 14:31:12.032000,0.0,11.42,0.00,11.1400,724308.0,8.148353e+06,inf,1.0,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-03 14:31:52.190000,0.0,11.42,0.00,11.0900,736572.0,8.155688e+06,inf,1.0,NaN,NaN,...,NaN,3.0,-3.305978e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-03 14:33:45.154000,0.0,11.42,0.00,11.1550,732244.0,8.146119e+06,inf,1.0,NaN,NaN,...,NaN,2.0,9.858033e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-03 14:35:03.433000,0.0,11.42,0.00,11.2090,727580.0,8.158092e+06,inf,1.0,NaN,NaN,...,NaN,2.0,4.507930e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-03 14:36:11.302000,0.0,11.42,0.00,11.2000,771128.0,8.651617e+06,inf,1.0,NaN,NaN,...,NaN,3.0,3.888771e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-08 19:54:40.227456,101.9,105.45,99.29,102.0800,502640.0,5.126439e+07,-0.002160,-1.0,34.519122,50.941412,...,172858.898742,1.0,8.943074e+07,100.0,34.977468,0.156072,0.167502,55.455382,-19.635582,-11.876824
2025-05-08 19:56:52.507584,101.9,105.45,99.29,101.6801,503700.0,5.128271e+07,-0.002063,-1.0,30.189645,50.863888,...,94843.554468,3.0,8.923341e+07,100.0,31.168217,0.146102,0.159721,52.608849,-22.939697,-13.457235
2025-05-08 19:58:34.941437,101.9,105.45,99.29,101.6900,504334.0,5.127041e+07,-0.001670,-1.0,30.422301,50.865741,...,18877.623553,1.0,8.923832e+07,100.0,27.832388,0.134253,0.150488,50.269033,-25.478334,-15.174534


In [ ]:
df=df[df.label_real.abs()>0.0001]

In [ ]:
features=[
 'RSI_14',
 'RSI_840',
 'RSI_70',
 'PPO_12_26_9',
 'PPOh_12_26_9',
 'PPOs_12_26_9',
 'PPO_40_200_12',
 'PPOh_40_200_12',
 'PPOs_40_200_12',
 'AROOND_14',
 'AROONU_14',
 'AROONOSC_14',
 'BOP',
 #'STOCHRSIk_14_14_3_3',
 #'STOCHRSId_14_14_3_3',
 'MFI_14',
 'MFI_70',
 #'VWAP_D',
 #'KVO_34_55_13',
 #'KVOs_34_55_13',
 #'ADX_14',
 #'DMP_14',
 #'DMN_14',
 #'ADX_70',
 #'DMP_70',
 #'DMN_70',
 #'ADX_840',
 #'DMP_840',
 #'DMN_840',
 'NATR_14',
 'NATR_70',
 'NATR_840',
 #'SQZPRO_20_2.0_20_2_1.5_1',
 #'SQZPRO_ON_WIDE',
 #'SQZPRO_ON_NORMAL',
 #'SQZPRO_ON_NARROW',
 #'SQZPRO_OFF',
 #'SQZPRO_NO',
 #'ADOSC_3_10',
 #'PVR',
 #'PVT',
 #'CHOP_14_1_100',
 #'RSX_14',
 #'RVGI_14_4',
 #'RVGIs_14_4',
 'UO_7_14_28',
 #'TSI_13_25_13',
 #'TSIs_13_25_13'
 ]

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

# Assuming 'df' is your DataFrame with a DateTimeIndex
# and 'features' is a list of your feature columns

# Create a TimeSeriesSplit object
# For example, to create 5 splits:
tscv = TimeSeriesSplit(n_splits=10)

# Iterate through the splits
for i,(train_index, test_index) in enumerate(tscv.split(df[features])):
    if i<9: continue
    X_train, X_test = df[features].iloc[train_index], df[features].iloc[test_index]
    y_train, y_test = df["label"].iloc[train_index], df["label"].iloc[test_index]

        # Create CatBoost classifier with random parameters
    from catboost import CatBoostClassifier

    # Create a CatBoost classifier
    model = CatBoostClassifier(
        iterations=1000,
        learning_rate=0.003,
        depth=4,
        loss_function="Logloss",
        #colsample_bylevel=0.3,
        rsm=0.3,
        random_strength=1.0,  # Increase for more randomness
        # bagging_temperature=1.0,  # Remove or comment out this line
        subsample=0.1,  # Use a subset of data for each iteration
        #bootstrap_type="Bayesian",  # Change to 'Bayesian' if you want to use bagging_temperature
        random_seed=42,  # Set random seed for reproducibility
        od_type="IncR",  # Overfitting detector
        od_wait=20,  # Number of iterations to wait before stopping
        verbose=50,  # Print progress every 50 iterations
    )

    # Fit the model
    model.fit(
        X_train,
        y_train,
        eval_set=(X_test, y_test),
        early_stopping_rounds=50,  # Stop if no improvement for 50 rounds
    )
    # Get confusion matrix
    from sklearn.metrics import confusion_matrix
    import seaborn as sns
    import matplotlib.pyplot as plt
    y_prob = model.predict_proba(X_test)[:, 1]
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Down', 'Up'], yticklabels=['Down', 'Up'])

    # Now you have X_train, X_test, y_train, y_test for each split
    # You can train your model on X_train, y_train and evaluate on X_test, y_test

0:	learn: 0.6882552	test: 0.6886806	best: 0.6886806 (0)	total: 219ms	remaining: 3m 38s
50:	learn: 0.5273977	test: 0.5334036	best: 0.5334036 (50)	total: 3.95s	remaining: 1m 13s
100:	learn: 0.4200110	test: 0.4304014	best: 0.4304014 (100)	total: 8.35s	remaining: 1m 14s


KeyboardInterrupt: 

In [ ]:
model.get_feature_importance(prettified=True)

### Save Model

In [ ]:
model.save_model("/content/drive/MyDrive/catboost_model.cbm")

### Vizualize Prediction

In [ ]:
res = df['close'].iloc[test_index].to_frame('price')
res['prob'] = y_prob


In [ ]:
res['log_future_return'] = np.log(res['price'].shift(-1)/res['price'])
np.exp(res[res.prob>0.5].log_future_return.sum())

0.7197196228052474

In [ ]:
res

,price,prob,log_future_return
ts_end,,,
2024-07-08 13:44:20.602776,175.6790,0.983753,0.001314
2024-07-08 13:47:25.223312,175.9100,0.984210,0.005160
2024-07-08 13:50:56.653716,176.8200,0.985220,-0.009547
2024-07-08 13:53:07.341924,175.1400,0.983960,-0.007968
2024-07-08 13:54:29.027090,173.7500,0.982165,0.010022
...,...,...,...
2025-05-08 19:52:08.159384,102.0100,0.590651,0.000686
2025-05-08 19:54:40.227456,102.0800,0.676099,-0.003925
2025-05-08 19:56:52.507584,101.6801,0.287362,0.000097


In [ ]:
res.log_future_return.sum()

-0.5463362213317275

### Feature Importance Analysis

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from catboost import CatBoostClassifier
import numpy as np

# Assuming 'df' is your DataFrame with a DateTimeIndex
# and 'features' is a list of your feature columns

# Create a TimeSeriesSplit object
tscv = TimeSeriesSplit(n_splits=2)

# To record selected features for each fold
selected_features_per_fold = []

# Iterate through the splits
for fold, (train_index, test_index) in enumerate(tscv.split(df[features])):
    print(f"\n=== Fold {fold + 1} ===")
    X_train, X_test = df[features].iloc[train_index], df[features].iloc[test_index]
    y_train, y_test = df["label"].iloc[train_index], df["label"].iloc[test_index]

    current_features = features.copy()

    # Recursive Feature Elimination (remove 1 feature at a time)
    while len(current_features) > 5:  # You can set a lower limit of features
        # Train model on current features
        model = CatBoostClassifier(
            iterations=300,
            learning_rate=0.01,
            depth=3,
            loss_function="Logloss",
            rsm=0.5,
            subsample=0.5,
            random_seed=42,
            verbose=0
        )
        model.fit(X_train[current_features], y_train)

        # Get feature importances
        importances = model.get_feature_importance()
        feature_importance_dict = dict(zip(current_features, importances))

        # Find the least important feature
        least_important_feature = min(feature_importance_dict, key=feature_importance_dict.get)
        print(f"Removing least important feature: {least_important_feature}")

        # Remove that feature
        current_features.remove(least_important_feature)

    print(f"Selected features for fold {fold + 1}: {current_features}")
    selected_features_per_fold.append(current_features.copy())

    # Now train your final model on the selected features
    final_model = CatBoostClassifier(
        iterations=1000,
        learning_rate=0.003,
        depth=4,
        loss_function="Logloss",
        rsm=0.3,
        subsample=0.1,
        random_strength=1.0,
        random_seed=42,
        od_type="IncR",
        od_wait=20,
        verbose=50,
    )

    final_model.fit(
        X_train[current_features],
        y_train,
        eval_set=(X_test[current_features], y_test),
        early_stopping_rounds=50,
    )

# After loop
print("\nSummary of selected features per fold:")
for i, feats in enumerate(selected_features_per_fold, 1):
    print(f"Fold {i}: {feats}")




=== Fold 1 ===
Removing least important feature: NATR_70
Removing least important feature: PPOh_12_26_9
Removing least important feature: NATR_840
Removing least important feature: PPOs_40_200_12
Removing least important feature: RSI_840
Removing least important feature: PPO_40_200_12
Removing least important feature: AROONOSC_14


KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from catboost import CatBoostClassifier
import numpy as np
from sklearn.metrics import roc_auc_score

tscv = TimeSeriesSplit(n_splits=6)
selected_features_per_fold = []

for fold, (train_index, test_index) in enumerate(tscv.split(df[features])):
    print(f"\n=== Fold {fold + 1} ===")
    X_train, X_test = df[features].iloc[train_index], df[features].iloc[test_index]
    y_train, y_test = df["label"].iloc[train_index], df["label"].iloc[test_index]

    current_features = features.copy()

    performance_history = []
    feature_history = []

    while len(current_features) > 2:  # Stop when very few features are left
        # Train model
        model = CatBoostClassifier(
            iterations=300,
            learning_rate=0.01,
            depth=8,
            loss_function="Logloss",
            rsm=0.5,
            subsample=0.5,
            random_seed=42,
            verbose=0
        )
        model.fit(X_train[current_features], y_train)

        # Evaluate on validation set
        val_preds = model.predict_proba(X_test[current_features])[:, 1]
        val_score = roc_auc_score(y_test, val_preds)  # or accuracy_score, etc.

        performance_history.append(val_score)
        feature_history.append(current_features.copy())

        # Get importances
        importances = model.get_feature_importance()
        feature_importance_dict = dict(zip(current_features, importances))

        # Remove least important feature
        least_important_feature = min(feature_importance_dict, key=feature_importance_dict.get)
        print(f"Removing least important feature: {least_important_feature}")
        current_features.remove(least_important_feature)

    # After elimination, find best feature set
    best_idx = np.argmax(performance_history)
    best_features = feature_history[best_idx]

    print(f"Best number of features for fold {fold + 1}: {len(best_features)}")
    print(f"Selected features: {best_features}")
    selected_features_per_fold.append(best_features.copy())

    # Train final model on best features
    final_model = CatBoostClassifier(
        iterations=1000,
        learning_rate=0.003,
        depth=4,
        loss_function="Logloss",
        rsm=0.3,
        subsample=0.1,
        random_strength=1.0,
        random_seed=42,
        od_type="IncR",
        od_wait=20,
        verbose=50,
    )
    final_model.fit(
        X_train[best_features],
        y_train,
        eval_set=(X_test[best_features], y_test),
        early_stopping_rounds=50,
    )


In [ ]:
features = [f for features in selected_features_per_fold for f in features]

In [ ]:
from collections import Counter
feature_counts = Counter(features)

In [ ]:
common_features = [feature for feature, count in feature_counts.most_common()]


In [ ]:
from pandas_ta.volume import kvo
kvo

In [ ]:
from pandas_ta.trend import chop
chop